In [1]:
import requests
import pandas as pd

In [2]:

url = 'https://datausa.io/api/data?drilldowns=State&measures=Population'


x = requests.get(url)
json_data = x.json()['data']
# json_data

In [3]:
# df = pd.read_json(json_data, orient ='records')
# df = pd.DataFrame.from_dict(json_data)
df = pd.DataFrame(json_data)

df.head(100)

,ID State,State,ID Year,Year,Population,Slug State
0,04000US01,Alabama,2019,2019,4903185,alabama
1,04000US02,Alaska,2019,2019,731545,alaska
2,04000US04,Arizona,2019,2019,7278717,arizona
3,04000US05,Arkansas,2019,2019,3017804,arkansas
4,04000US06,California,2019,2019,39512223,california
...,...,...,...,...,...,...
95,04000US48,Texas,2018,2018,28701845,texas
96,04000US49,Utah,2018,2018,3161105,utah
97,04000US50,Vermont,2018,2018,626299,vermont
98,04000US51,Virginia,2018,2018,8517685,virginia


In [4]:
years = df.Year.unique()[::-1]
years

array(['2013', '2014', '2015', '2016', '2017', '2018', '2019'],
      dtype=object)

In [5]:
states = df.State.unique()
print(states)

['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'
 'Connecticut' 'Delaware' 'District of Columbia' 'Florida' 'Georgia'
 'Hawaii' 'Idaho' 'Illinois' 'Indiana' 'Iowa' 'Kansas' 'Kentucky'
 'Louisiana' 'Maine' 'Maryland' 'Massachusetts' 'Michigan' 'Minnesota'
 'Mississippi' 'Missouri' 'Montana' 'Nebraska' 'Nevada' 'New Hampshire'
 'New Jersey' 'New Mexico' 'New York' 'North Carolina' 'North Dakota'
 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania' 'Rhode Island' 'South Carolina'
 'South Dakota' 'Tennessee' 'Texas' 'Utah' 'Vermont' 'Virginia'
 'Washington' 'West Virginia' 'Wisconsin' 'Wyoming' 'Puerto Rico']


In [6]:
df.loc[(df['Year'] == '2013') & (df['State'] == 'Alabama')].values[0][4]


4833722

In [7]:
def prime_factors(n):
    i = 2
    factors = []
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors.append(str(i))
    if n > 1:
        factors.append(str(n))
    return factors

csv_columns = ['State Name', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2019 Factors']
data = []
for state in states:
    row = []
    row.append(state)
    for year in years:
        curr_pop = df.loc[(df['Year'] == year) & (df['State'] == state)].values[0][4]
        if year == '2013':
            row.append(curr_pop)
        else:
            prev_pop = df.loc[(df['Year'] == str(int(year)-1)) & (df['State'] == state)].values[0][4]
            percentage = (curr_pop - prev_pop) * 100/ prev_pop 
            per_string = '(' + str(round(percentage, 2)) + '% )'
            row.append(str(curr_pop) + per_string)
    prime_fs = prime_factors(df.loc[(df['Year'] == '2019') & (df['State'] == state)].values[0][4])
    prime_factors_str = ', '.join(prime_fs)
    row.append(prime_factors_str)

    data.append(row)

In [8]:
df = pd.DataFrame(data,columns=csv_columns)


In [9]:
df.head()

,State Name,2013,2014,2015,2016,2017,2018,2019,2019 Factors
0,Alabama,4833722,4849377(0.32% ),4858979(0.2% ),4863300(0.09% ),4874747(0.24% ),4887871(0.27% ),4903185(0.31% ),"3, 5, 7, 7, 7, 953"
1,Alaska,735132,736732(0.22% ),738432(0.23% ),741894(0.47% ),739795(-0.28% ),737438(-0.32% ),731545(-0.8% ),"5, 146309"
2,Arizona,6626624,6731484(1.58% ),6828065(1.43% ),6931071(1.51% ),7016270(1.23% ),7171646(2.21% ),7278717(1.49% ),"3, 1019, 2381"
3,Arkansas,2959373,2966369(0.24% ),2978204(0.4% ),2988248(0.34% ),3004279(0.54% ),3013825(0.32% ),3017804(0.13% ),"2, 2, 754451"
4,California,38332521,38802500(1.23% ),39144818(0.88% ),39250017(0.27% ),39536653(0.73% ),39557045(0.05% ),39512223(-0.11% ),"3, 3, 4390247"


In [10]:
df.to_csv('output.csv', index=False)